In [1]:
import pandas as pd
import numpy as np

In [2]:
optis = pd.read_csv("../raw-data/OpTIS/predictors_2020_04_01_shp.csv", low_memory=False)

In [3]:
optis = optis.set_index('row_id')

In [4]:
# Focus on pre-2012 data
optis_pre2012 = optis.query('Year < 2012')

In [5]:
a = optis_pre2012.groupby(['State_Name','County_Name'])['Corn_RedTill_High_Pct'].mean().reset_index().rename(columns={'Corn_RedTill_High_Pct': 'meanRT'})

In [6]:
# Get lowest and highest 5 within each state
low = a.sort_values('meanRT', ascending=True).groupby(['State_Name']).head(5).reset_index()
high = a.sort_values('meanRT', ascending=False).groupby(['State_Name']).head(5).reset_index()

In [7]:
# Select out Minnesota and Kansas
states = ['Kansas', 'Minnesota']
selectCounties = low[low.State_Name.isin(states)].append(high[high.State_Name.isin(states)],ignore_index=True)

In [23]:
# Merge with existing data to be able to get FIPS codes
merged = selectCounties.merge(optis[['State_Name','County_Name','County_FIPS']], 
                              on=['State_Name','County_Name'], 
                              how='left').drop_duplicates()

     index State_Name      County_Name     meanRT  County_FIPS
0      392  Minnesota        Sherburne  13.271429        27141
14     293     Kansas            Allen  15.557143        20001
28     383  Minnesota       Pennington  20.128571        27113
42     302     Kansas         Crawford  20.757143        20037
56     405  Minnesota           Winona  21.214286        27169
70     305     Kansas              Elk  24.728571        20049
84     402  Minnesota       Washington  24.785714        27163
98     298     Kansas            Chase  25.000000        20017
112    300     Kansas         Cherokee  25.000000        20021
126    356  Minnesota           Dakota  26.328571        27037
140    318     Kansas           Morris  81.057143        20127
154    391  Minnesota            Scott  68.671429        27139
168    369  Minnesota         Le Sueur  64.528571        27079
182    297     Kansas            Brown  63.085714        20013
196    407  Minnesota  Yellow Medicine  61.642857      

In [24]:
# Export file to run in GEE
merged.to_csv('../processed-data/counties-for-gee.csv', index=False)